In [ ]:
# %matplotlib notebook

import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, Polygon

import matplotlib
norm = matplotlib.colors.Normalize()

%matplotlib inline

from shapely.geometry import box, Polygon

land = gpd.read_file('data/gshhs/f/GSHHS_f_L1.shp')

# 3.01

## LEBANON

In [ ]:
path = 'cases/lebanon_remake/out_files/MDK_SIM_2006_07_13_0800_lebanon_remake/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

lon_min,lon_max = 35,36
lat_min,lat_max = 33,34.5

rec = land.cx[lon_min:lon_max, lat_min:lat_max]

for t in range(0,480):
# t = 1
    dss = ds.isel(time=t)

    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_facecolor('#ADD8E6')
    rec.plot(ax=ax,color="#FFFDD0", edgecolor='black', zorder = 1000)
    ax.scatter(dss.longitude,dss.latitude,dss.non_evaporative_volume,color='r')
    
    plt.xlim(lon_min,lon_max)
    plt.ylim(lat_min,lat_max)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.title(f'time={t}')

    plt.grid()

    plt.savefig(f'{path}/figures/fig_test_{t:03d}.png',dpi=50)

### PARIA

In [ ]:
path = '/work/opa/ia19223/Medslik-II/cases/paria/out_files/MDK_SIM_2017_04_24_1300_paria_casestudy/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

lon_min,lon_max = -64.1,-63.5
lat_min,lat_max = 10.6,11

rec = land.cx[lon_min:lon_max, lat_min:lat_max]

for t in range(0,48):
# t = 1
    dss = ds.isel(time=t)

    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_facecolor('#ADD8E6')
    rec.plot(ax=ax,color="#FFFDD0", edgecolor='black', zorder = 1000)
    ax.scatter(dss.longitude,dss.latitude,dss.non_evaporative_volume,color='r')
    
    plt.xlim(lon_min,lon_max)
    plt.ylim(lat_min,lat_max)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.title(f'time={t}')

    plt.grid()

    plt.savefig(f'{path}/figures/fig_test_{t:03d}.png',dpi=50)

### PARIA REMAKE

In [ ]:
path = '/work/opa/ia19223/Medslik-II/cases/paria_remake/out_files/MDK_SIM_2017_04_24_1300_paria_remake/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

lon_min,lon_max = -64.1,-63.5
lat_min,lat_max = 10.6,11

rec = land.cx[lon_min:lon_max, lat_min:lat_max]

for t in range(0,48):
# t = 1
    dss = ds.isel(time=t)

    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_facecolor('#ADD8E6')
    rec.plot(ax=ax,color="#FFFDD0", edgecolor='black', zorder = 1000)
    ax.scatter(dss.longitude,dss.latitude,dss.non_evaporative_volume,color='r')
    
    plt.xlim(lon_min,lon_max)
    plt.ylim(lat_min,lat_max)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.title(f'time={t}')

    plt.grid()

    plt.savefig(f'{path}/figures/fig_test_{t:03d}.png',dpi=50)

### Test in Sicilia

In [ ]:
path = '/work/opa/ia19223/Medslik-II/MEDSLIK_II_3.01/OUT/MDK_SIM_2022_04_28_1300_South_Sicilia/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

lon_min,lon_max = 13,14
lat_min,lat_max = 36,37

rec = land.cx[lon_min:lon_max, lat_min:lat_max]

for t in range(0,48):
# t = 1
    dss = ds.isel(time=t)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_facecolor('#ADD8E6')
    rec.plot(ax=ax,color="#FFFDD0", edgecolor='black', zorder = 1000,aspect=1)
    ax.scatter(dss.longitude,dss.latitude,dss.non_evaporative_volume,color='r')
    
    plt.xlim(lon_min,lon_max)
    plt.ylim(lat_min,lat_max)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.title(f'time={t}')

    plt.grid()

    plt.savefig(f'{path}/figures/fig_test_{t:03d}.png',dpi=50)

# Test in Taranto

## WITOIL

In [ ]:
path = '/work/opa/ia19223/Medslik-II/cases/taranto_witoil/witoil/OUT/MERCATOR_global_2023_11_23_1300_/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

ds

In [ ]:
path = '/work/opa/ia19223/Medslik-II/cases/taranto_witoil/witoil/OUT/MERCATOR_global_2023_11_23_1300_/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

lon_min,lon_max = 17,17.5
lat_min,lat_max = 40,40.7

rec = land.cx[lon_min:lon_max, lat_min:lat_max]

for t in range(0,24):
# t = 1
    dss = ds.isel(time=t)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_facecolor('#ADD8E6')
    rec.plot(ax=ax,color="#FFFDD0", edgecolor='black', zorder = 1000,aspect=1)
    ax.scatter(dss.longitude,dss.latitude,dss.non_evaporative_volume,color='r')
    
    plt.xlim(lon_min,lon_max)
    plt.ylim(lat_min,lat_max)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.title(f'time={t}')

    plt.grid()

    plt.savefig(f'{path}/figures/fig_test_{t:03d}.png',dpi=50)

## MANUAL

In [ ]:
path = '/work/opa/ia19223/Medslik-II/cases/taranto_manual/out_files/MDK_SIM_2023_11_23_1300_taranto_manual/'
filename = path + 'spill_properties.nc'
ds = xr.open_dataset(filename)

lon_min,lon_max = 17,17.5
lat_min,lat_max = 40,40.7

rec = land.cx[lon_min:lon_max, lat_min:lat_max]

for t in range(0,24):
# t = 1
    dss = ds.isel(time=t)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.set_facecolor('#ADD8E6')
    rec.plot(ax=ax,color="#FFFDD0", edgecolor='black', zorder = 1000,aspect=1)
    ax.scatter(dss.longitude,dss.latitude,dss.non_evaporative_volume,color='r')
    
    plt.xlim(lon_min,lon_max)
    plt.ylim(lat_min,lat_max)

    plt.xlabel('Longitude')
    plt.ylabel('Latitude')

    plt.title(f'time={t}')

    plt.grid()

    plt.savefig(f'{path}/figures/fig_test_{t:03d}.png',dpi=50)

# Plotting Ascii files

## Bath Files

In [ ]:
def plot_bath(path):
    raw = pd.read_csv(path)
    #coordinates
    temp = raw.iloc[0,0].split(' ')
    coords = [float(value) if value else 0 for value in temp]
    coords = coords[:-1]

    lat_min, lat_max = coords[0],coords[1]
    lon_min, lon_max = coords[2],coords[3]

    #grid elements
    temp = raw.iloc[1,0].split(' ')
    elements = [int(value) if value else 0 for value in temp]
    elements[:-1]

    y_elem, x_elem = elements[0], elements[1]

    y = np.linspace(lat_min,lat_max,y_elem)
    x = np.linspace(lon_max,lon_min,x_elem)

    xx,yy = np.meshgrid(y,x)

    values = pd.read_csv(path,skiprows=2)

    #reshaped matrix
    reshaped = values.values.squeeze().reshape((x_elem,y_elem))

    dep = np.ma.masked_where(reshaped > 8000,reshaped)

    plt.figure()
    plt.pcolormesh(xx,yy,dep)
    plt.colorbar()


In [ ]:
# %matplotlib widget

In [ ]:
plot_bath('cases/taranto_manual/bnc_files/dtm.bath')

In [ ]:
plot_bath('cases/taranto_witoil/witoil/DTM_INP/vnzl_.bath')

## Map files

In [ ]:
def plot_map(file_path):
    # Read the file, skipping lines ending with 0

    columns = ['Longitude', 'Latitude']
    df = pd.read_csv(file_path, delim_whitespace=True, header=None, names=columns,skiprows=1)

    df['label'] = np.NaN
    df['Latitude'] = df['Latitude'].replace(0, np.nan)

    # Initialize counter variable for labels
    label_counter = 1

    # Iterate through rows and update 'label' when 'Latitude' is NaN
    for index, row in df.iterrows():
        if pd.isna(row['Latitude']):
            df.at[index, 'label'] = label_counter
            label_counter += 1

    df['label'] = df.label.ffill()

    df = df.dropna()

    # # Create a GeoDataFrame with LineString geometries
    geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]

    gdf = gpd.GeoDataFrame(df,geometry=geometry)

    df_pol = gdf.groupby('label')['geometry'].apply(lambda x: Polygon(x.tolist())).reset_index()

    # Declare the result as a new a GeoDataFrame
    gdf_pol = gpd.GeoDataFrame(df_pol, geometry = 'geometry')

    gdf_pol.plot()


In [ ]:
%matplotlib widget
# %matplotlib inline

In [ ]:
plot_map('/work/opa/ia19223/Medslik-II/cases/taranto_manual/bnc_files/dtm.map')

In [ ]:
plot_map('/work/opa/ia19223/Medslik-II/cases/taranto_witoil/witoil/DTM_INP/vnzl.map')


## Mrc files